# Gestion des membre CCUdeM

Exécuter les cellules dans l'ordre. La cellule suivante contient les fonctions qui serviront à la gestion des membres. 

Les cellules à la fin servent à mettre à jour les comptes des membres. 


In [44]:
import hashlib
import pandas as pd
import uuid
import json

# duplicate
def format_name(first_name: str, last_name: str) -> str:
    """
    Format the name of a person
    :param first_name: first name
    :param last_name: last name
    :return: formatted name
    """
    return f'{first_name} {last_name}'


def __date_str_to_datetime(date_str: str) -> pd.Timestamp:
    """
    :param date_str: str in the format 'YYYY-MM-DD'
    :return: pd.Timestamp
    """
    return pd.Timestamp(date_str)


def __i2hex(i: str) -> str:
    """
    :param i: str integer
    :return: str hex representation of the integer
    """
    return hex(int(i))


# Hashing functions

def __salt():
    """
    Salt to be kept secret on the server
    Has to be unique for each member
    :return: str of 12 characters
    """
    return uuid.uuid4().hex[:12]

def __hash(member_id, name, surname, birth_year, expiration_date, salt) -> str:
    """
    :param member_id: str
    :param name: str
    :param surname: str
    :param birth_year: str
    :param expiration_date: str
    :param salt: str (uuid to be kept secret on the server)
    """
    birth_year = str(__i2hex(birth_year))
    member_str = f'{int(member_id)}{name}{birth_year}{surname}{expiration_date}{salt}'
    return hashlib.sha1(member_str.encode()).hexdigest()[:12]

def __hash_member(member: pd.Series) -> str:
    """
    :param member: pd.Series with the following columns:
    - member_id: str
    - first_name: str
    - last_name: str
    - birth_year: str
    - expiration_date: str
    - salt: str (uuid to be kept secret on the server)
    :return: public hash of the member (str)
    """
    member_id = member['member_id']
    name = member['first_name']
    surname = member['last_name']
    birth_year = str(member['birth_year'])[2:]
    expiration_date = member['expiration_date'].replace('-', '')
    salt = member['salt']
    return __hash(member_id, name, surname, birth_year, expiration_date, salt)


# Get Member functions

def get_member_by_property(members: pd.DataFrame, property: str, value: str) -> pd.Series | None:
    """
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    - first_name: str
    - last_name: str
    - birth_year: str
    - join_date: str
    - expiration_date: str
    - salt: str (uuid to be kept secret on the server)
    - hash: str (public hash of the member)
    :param property: str (column name)
    :param value: str (value to search for)
    :return: pd.Series with the member data or None if not found
    """
    member = members[members[property] == value]
    member = member.iloc[0] if not member.empty else None
    return member

def get_member_by_id(members: pd.DataFrame, member_id: str) -> pd.Series | None:
    """
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    :member_id: str id of the searched member
    :return: pd.Series of the member or None
    """
    return get_member_by_property(members, 'member_id', member_id)

def get_member_by_hash(members: pd.DataFrame, member_hash: str) -> pd.Series:
    """
    :param members: pd.DataFrame with the following columns:
    - hash: str
    :param member_hash: str hash of the member
    :return: pd.Series of the member or None
    """
    return get_member_by_property(members, 'hash', member_hash)

def get_member(members: pd.DataFrame, first_name: str, last_name: str, birth_year: str) -> pd.Series:
    """
    :param members: pd.DataFrame with the following columns:
    - name: str
    - surname: str
    - birth_year: str
    :param name: str
    :param surname: str
    :param birth_year: str
    :return: pd.Series of the member or None
    """
    member = members[
        (members['first_name'] == first_name) & 
        (members['last_name'] == last_name) & 
        (members['birth_year'].astype(str) == str(birth_year))
    ]
    member = member.iloc[0] if not member.empty else None
    return member


# Get Member ID function

def get_member_id(members: pd.DataFrame, first_name: str, last_name: str, birth_year: str) -> str:
    """
    :param members: pd.DataFrame with the following columns:
    - name: str
    - surname: str
    - birth_year: str
    :param name: str
    :param surname: str
    :param birth_year: str
    :return: str member_id of the member or None
    """
    member = get_member(members, first_name, last_name, birth_year)
    return member['member_id'] if member is not None else None


# Membership functions

def is_member_valid(members: pd.DataFrame, member_id: int, member_hash: str) -> bool:
    """
    Used to verify the member's card
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    - hash: str
    :param member: pd.Series with the following columns:
    - member_id: str
    - hash: str
    :return: bool True if the member is valid, False otherwise
    """
    m = get_member_by_id(members, member_id)
    if m is None:
        return False
    return member_hash == m['hash']

def is_member_active(members: pd.DataFrame, member_id: int) -> bool:
    """
    Tells if a member is active or not (based on the expiration date)
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    - expiration_date: str
    :param member_id: int
    :return: bool True if the member is active, False otherwise
    """
    member = get_member_by_id(members, member_id)
    if member is None:
        return False
    expiration_date = member['expiration_date']
    today = pd.Timestamp.now()
    return __date_str_to_datetime(expiration_date) >= today

def __extract_json_qr_code(qr_code: str) -> pd.Series | None:
    """
    :param qr_code: str
    :return: dict
    """
    qr_code = qr_code.replace('\'', '\"')
    try:
        json_qr = json.loads(qr_code)
        member_id = json_qr['CCUdeM']
        expiration_date = json_qr['exp']
        name = json_qr['nom']
        h = json_qr['h']
    except:
        return None
    return pd.Series({'member_id': member_id, 'expiration_date': expiration_date, 'name': name, 'hash': h})

def qr_code_status(members: pd.DataFrame, qr_code: str, verbose=True) -> str:
    """
    "{'CCUdeM': 1, 'exp': '2025-03-01', 'nom': 'Etienne Comtois', 'h': '1e8ee15d4506'}"
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    - hash: str
    - expiration_date: str
    - first_name: str
    - last_name: str
    :param qr_code: str
    :return: str (active, inactive, invalid:reason)
    """
    temp_member = __extract_json_qr_code(qr_code)
    if temp_member is None:
        return 'invalid:qr'

    member = get_member_by_id(members, temp_member['member_id'])
    if member is None:
        return 'invalid:not_found'
    
    if verbose:
        print(f'id:{member["member_id"]} name:{member["first_name"]} {member["last_name"]} exp:{member["expiration_date"]} hash:{member["hash"]}')

    # verify hash
    if not is_member_valid(members, temp_member['member_id'], temp_member['hash']):
        return 'invalid:hash'
    
    # verify same name
    name = format_name(member['first_name'], member['last_name'])
    if name != temp_member['name']:
        return 'invalid:name'
    
    # verify same expiration date
    exp_date = member['expiration_date']
    if exp_date != temp_member['expiration_date']:
        return 'invalid:expiration'
    
    # verify if active
    if is_member_active(members, temp_member['member_id']):
        return 'active'
    return 'inactive'



# Add Member functions

def __replace_member(members: pd.DataFrame, member: pd.Series) -> pd.DataFrame:
    """
    Replace a member - used for updating its informations
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    :param member: pd.Series with the following columns:
    - member_id: str
    :return: pd.DataFrame with the updated member
    """
    members = members[members['member_id'] != member['member_id']]
    members = pd.concat([members, member.to_frame().T], axis=0)
    members = members.sort_values(by='member_id')
    return members

def add_new_member(members: pd.DataFrame, name, surname, birth_year, join_date) -> pd.DataFrame:
    m = get_member(members, name, surname, birth_year)
    if m is not None:
        return members
    
    member_id = members.shape[0] + 1
    m = pd.Series({
        'member_id': member_id,
        'first_name': name,
        'last_name': surname,
        'birth_year': birth_year,
        'join_date': join_date,
        'expiration_date': join_date,
        'salt': __salt()
    })
    m = m.to_frame().T
    m['hash'] = __hash_member(m)
    members = pd.concat([members, m], axis=0)
    return members

def renew_membership(members: pd.DataFrame, member_id: int, expiration_date: str) -> pd.DataFrame:
    """
    Renew the membership of an existing member
    :param members: pd.DataFrame with the following columns:
    - member_id: str
    - expiration_date: str
    - hash: str
    :param member_id: int
    :param expiration_date: str
    :return: pd.DataFrame with the updated member
    """

    m = get_member_by_id(members, member_id)
    if m is None:
        return members
    m['expiration_date'] = expiration_date
    m['hash'] = __hash_member(m)
    members = __replace_member(members, m)
    return members



# Ajouter un nouveau membre & Renouveller membership

Le code suivant permet de mettre à jour le membership. Le membre sera ajouté si il n'existe pas

In [46]:
# Valeurs à changer - information sur le membre
nouvelle_expriation = '2024-03-01' 
prenom = 'Philippe'
nom = 'Dessureault'
annee_naissance = '2004'

# Code pour renouveler l'adhésion
members = pd.read_csv('members.csv')
member = get_member(members, prenom, nom, annee_naissance)
if member is None:
    now = pd.Timestamp.now().strftime('%Y-%m-%d')
    members = add_new_member(members, prenom, nom, annee_naissance, now)
    member = get_member(members, prenom, nom, annee_naissance)
    print(f'Nouveau membre ajouté : {member["member_id"]}')
members = renew_membership(members, member['member_id'], nouvelle_expriation)
members.to_csv('members.csv', index=False)
print(f'Membre {member["member_id"]} renouvelé jusqu\'au {nouvelle_expriation}')

Membre 4 renouvelé jusqu'au 2024-03-01


# Vérification d'un Membre

In [47]:
# Valeur scanné du code QR
# value_qr = "{'CCUdeM': 1, 'exp': '2025-03-01', 'nom': 'Etienne Comtois', 'h': '1e8ee15d4506'}"
value_qr = "{'CCUdeM': 4, 'exp': '2024-03-01', 'nom': 'Philippe Dessureault', 'h': 'bf966524984e'}"

# Vérification du code QR
members = pd.read_csv('members.csv')
print(qr_code_status(members, value_qr))

id:4 name:Philippe Dessureault exp:2024-03-01 hash:bf966524984e
inactive
